<a href="https://colab.research.google.com/github/R-A-N-N/Final-Year-Project/blob/main/Twitter_Problem_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Authentication

In [79]:
import os
import tweepy as tw
from time import sleep
import pandas as pd
import csv 
import re 
import string
# import preprocessor as p

In [80]:
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

#please copy paste all the keys which are pinned on our discord server in back-end channel 

#Method 1 - Extraction

In [81]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [82]:
# Define the search term and the date_since date as variables
search_words = "India"
date_since = "2000-01-01"

In [83]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)
tweets


In [84]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(5)

# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)
    

RT @TimesNow: Islamophobia is a rational fear among Muslims to discuss Islam in its entirety. It is the so-called moderate Muslims who are…
RT @_timos_: India Bookkeeping App Raises $100 Million From Tribe, Moore https://t.co/Em8u1Wm8AN #digitalhealth #socialmedia #digitalmarket…
RT @UN_Women: Meet some of the women on the front lines of the #COVID19 response in India as the country emerges from the deadly second wav…
RT @prasanto: DoT extends suspension of official Ashish Joshi - who was suspended for complaining about Kapil Mishra's hate speech - a susp…
RT @Nitin_Rivaldo: When Jihadis led by SUFI Musliyar rip open pregnent women &amp; butchered Hindus

Gandhi praised them,"Brave God-fearing Mop…


In [85]:
# Collect tweets
tweets = tw.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since).items(5)

# Collect a list of tweets
[tweet.text for tweet in tweets]

['RT @TimesNow: Islamophobia is a rational fear among Muslims to discuss Islam in its entirety. It is the so-called moderate Muslims who are…',
 'RT @_timos_: India Bookkeeping App Raises $100 Million From Tribe, Moore https://t.co/Em8u1Wm8AN #digitalhealth #socialmedia #digitalmarket…',
 'RT @UN_Women: Meet some of the women on the front lines of the #COVID19 response in India as the country emerges from the deadly second wav…',
 "RT @prasanto: DoT extends suspension of official Ashish Joshi - who was suspended for complaining about Kapil Mishra's hate speech - a susp…",
 'RT @Nitin_Rivaldo: When Jihadis led by SUFI Musliyar rip open pregnent women &amp; butchered Hindus\n\nGandhi praised them,"Brave God-fearing Mop…']

#Method 2 - Extraction

In [86]:
search_words = "Thane"      #enter your words
new_search = search_words + " -filter:retweets"

In [87]:
csvFile = open('Tweets', 'a')
csvWriter = csv.writer(csvFile)

In [88]:
for tweet in tw.Cursor(api.search,q=new_search,count=100,
                           lang="en",
                           since_id=0).items():
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'),tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8')])


In [89]:
# extracted date , time , username , location ,tweet 
# Extracted 3945 Tweets 
# which we have to first manually classify for our training dataset.
# train Naive Bayes ML model